In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix, classification_report
from sklearn.impute import SimpleImputer


In [ ]:
# Memuat dataset
data = pd.read_csv('/content/panic_attack_dataset.csv')

In [ ]:
data.head()

In [ ]:
print(data.describe())

In [ ]:
#memeriksa nilai yang hilang
(data.isnull().sum())

In [ ]:
#mengganti nilai null menjadi 0
data = data.fillna(0)
#mengecek kolom yang sudah diganti nilai nullnya menjadi 0
data.isnull().sum()

# **EDA**

In [ ]:
# Distribusi Frekuensi Serangan Panik berdasarkan Gender
sns.countplot(x='Gender', hue='Panic_Attack_Frequency', data=data)
plt.title('Frekuensi Serangan Panik berdasarkan Gender')
plt.show()

In [ ]:
# Box Plot Durasi berdasarkan Gender
plt.figure(figsize=(10, 6))
sns.boxplot(x='Gender', y='Duration_Minutes', data=data)
plt.title('Box Plot Durasi dalam Menit berdasarkan Gender')
plt.xlabel('Gender')
plt.ylabel('Durasi (Menit)')
plt.show()

In [ ]:
# Histogram distribusi usia
plt.figure(figsize=(10, 6))
sns.histplot(data['Age'], bins=30, kde=True)
plt.title('Distribusi Usia')
plt.xlabel('Usia')
plt.ylabel('Frekuensi')
plt.show()

In [ ]:
# Membuat bar chart untuk visualisasi distribusi kesehatan mental mahasiswa
plt.figure(figsize=(10, 6))
sns.barplot(x='Condition', y='Frequency', data=pd.DataFrame({'Condition': ['Depresi', 'Kecemasan', 'Serangan Panik'],
                                                             'Frequency': [65, 66, 67]}))
plt.title('Distribusi Kondisi Kesehatan Mental Mahasiswa')
plt.xlabel('Kondisi Kesehatan Mental')
plt.ylabel('Frekuensi')
plt.show()

# **PRE-PROSECCING**

In [ ]:
# Mengimputasi nilai yang hilang untuk kolom kategorikal jika diperlukan
imputer = SimpleImputer(strategy='most_frequent')
data['Caffeine_Intake'] = imputer.fit_transform(data[['Caffeine_Intake']])

# Memisahkan data menjadi fitur dan variabel target untuk regresi
X = data.drop(['Panic_Attack_Frequency'], axis=1)
y = data['Panic_Attack_Frequency']

# Melakukan encoding pada variabel kategorikal menggunakan Label Encoding untuk fitur regresi
categorical_columns = X.select_dtypes(include=['object']).columns.tolist()
label_encoders = {}
for column in categorical_columns:
    le = LabelEncoder()
    X[column] = le.fit_transform(X[column].astype(str))
    label_encoders[column] = le

# Standarisasi set fitur
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


# **IMPLEMENTASI ALGORITMA RANDOM FOREST**

In [ ]:
# Memisahkan data menjadi set pelatihan dan pengujian untuk analisis regresi
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Menggunakan Random Forest Regressor untuk model regresi
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)



In [ ]:
# Melakukan prediksi pada set pengujian
y_pred = rf_model.predict(X_test)

# Mengevaluasi kinerja model regresi
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("=== Evaluasi Model Regresi ===")
print(f'Mean Squared Error(MSE): {mse:.2f}')
print(f'R^2 Score: {r2:.2f}')


# **SELEKSI FITUR**

In [ ]:
# Seleksi Fitur berdasarkan Feature Importance dari Random Forest
feature_importances = rf_model.feature_importances_
features = X.columns

# Membuat DataFrame untuk menampilkan fitur dan pentingnya
importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importances})
importance_df.sort_values(by='Importance', ascending=False, inplace=True)

# Menampilkan fitur yang paling penting
print(importance_df)

In [ ]:
# Visualisasi Feature Importance
plt.figure(figsize=(12, 6))
sns.barplot(x='Importance', y='Feature', data=importance_df)
plt.title('Feature Importance dari Model Random Forest')
plt.show()

# **VISUALISASI HASIL PREDIKSI**

In [ ]:
# Visualisasi hasil prediksi dibandingkan dengan nilai aktual
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred)
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=2)
plt.xlabel('Nilai Aktual')
plt.ylabel('Prediksi')
plt.title('Prediksi vs Nilai Aktual')
plt.show()


# **KLASIFIKASI DAN CONFUSION MATRIX**

In [ ]:
# Membuat variabel target biner berdasarkan frekuensi serangan panik.
y_classification = (data['Panic_Attack_Frequency'] > 5).astype(int)  # Contoh threshold

# Memisahkan data menjadi set pelatihan dan pengujian untuk analisis klasifikasi
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X, y_classification, test_size=0.2, random_state=42)

# Menggunakan Logistic Regression sebagai contoh model klasifikasi
from sklearn.linear_model import LogisticRegression

classifier_model = LogisticRegression(max_iter=1000)
classifier_model.fit(X_train_class, y_train_class)

# Melakukan prediksi pada set pengujian untuk klasifikasi
y_pred_class = classifier_model.predict(X_test_class)

# Mengevaluasi kinerja model klasifikasi dengan confusion matrix
confusion_mat_class = confusion_matrix(y_test_class, y_pred_class)
print('Confusion Matrix:\n', confusion_mat_class)

class_report_class = classification_report(y_test_class, y_pred_class)
print('Classification Report:\n', class_report_class)

# Visualisasi Heatmap untuk Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_mat_class, annot=True, fmt='d', cmap='Blues', xticklabels=['Tidak Panic', 'Panic'], yticklabels=['Tidak Panic', 'Panic'])
plt.title('Heatmap Confusion Matrix')
plt.xlabel('Label Prediksi')
plt.ylabel('Label Aktual')
plt.show()

